# Capstone Project

# (Crimes in India)

# Phase 3

# SQL Operations









3.1	Insert records from 42_District_wise_crimes_committed_against_women_2001_2012.csv into a table

In [19]:
from pyspark.sql import SparkSession

In [20]:
spark=SparkSession.builder.appName('Crimes').getOrCreate()

In [21]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CreateTableAndInsertData") \
    .getOrCreate()

# Define the schema
schema = """
    State_UT VARCHAR(255),
    District VARCHAR(255),
    Year INT,
    Rape INT,
    Kidnapping INT,
    Dowry_Deaths INT,
    Assault_on_women_with_intent_to_outrage_her_modesty INT,
    insult_to_modesty_of_women INT,
    Cruelty_by_Husband_or_his_Relatives INT,
    Importation_of_Girls INT
"""

# Read data from CSV
data_path = "file:///C:/Users/Sayan Mondal/Downloads/India_Crime - Copy/India_Crime - Copy/42_District_wise_crimes_committed_against_women_2001_2012.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Print the first few rows of the DataFrame
print("First few rows of the DataFrame:")
df.show(5)

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("crime_data")

df.show()

First few rows of the DataFrame:
+--------------+-------------+----+----+------------------------+------------+---------------------------------------------------+--------------------------+-----------------------------------+--------------------+
|      STATE/UT|     DISTRICT|Year|Rape|Kidnapping and Abduction|Dowry Deaths|Assault on women with intent to outrage her modesty|Insult to modesty of Women|Cruelty by Husband or his Relatives|Importation of Girls|
+--------------+-------------+----+----+------------------------+------------+---------------------------------------------------+--------------------------+-----------------------------------+--------------------+
|ANDHRA PRADESH|     ADILABAD|2001|  50|                      30|          16|                                                149|                        34|                                175|                   0|
|ANDHRA PRADESH|    ANANTAPUR|2001|  23|                      30|           7|                             

In [4]:
df.columns

['STATE/UT',
 'DISTRICT',
 'Year',
 'Rape',
 'Kidnapping and Abduction',
 'Dowry Deaths',
 'Assault on women with intent to outrage her modesty',
 'Insult to modesty of Women',
 'Cruelty by Husband or his Relatives',
 'Importation of Girls']

3.2	Write SQL query to find the highest number of rapes & Kidnappings that happened in which state, District, and year

In [22]:
from pyspark.sql.functions import col, max
# Find the maximum number of rapes and kidnappings
max_values = df.selectExpr("max(Rape) as max_rape", "max(`Kidnapping and Abduction`) as max_kidnapping").first()

# Extract the maximum values
max_rape = max_values["max_rape"]
max_kidnapping = max_values["max_kidnapping"]

# Filter the DataFrame to get the corresponding state, district, and year
result = df.filter((col("Rape") == max_rape) | (col("Kidnapping and Abduction") == max_kidnapping)) \
           .select("STATE/UT", "DISTRICT", "Year", "Rape", "Kidnapping and Abduction")

# Show the result
result.show()

+--------------+--------+----+----+------------------------+
|      STATE/UT|DISTRICT|Year|Rape|Kidnapping and Abduction|
+--------------+--------+----+----+------------------------+
|MADHYA PRADESH|   TOTAL|2012|3425|                    1127|
| UTTAR PRADESH|   TOTAL|2012|1963|                    7910|
+--------------+--------+----+----+------------------------+



3.3	Write SQL query to find All the lowest number of rapes & Kidnappings that happened in which state, District, and year

In [23]:
from pyspark.sql.functions import col, min

min_murders = df.select(min(col("Rape"))).first()[0]

# Filter the DataFrame to get rows where murders are at the minimum
lowest_murder_districts = df.filter(col("Rape") == min_murders)

# Show the result
lowest_murder_districts.show()

+-----------------+-----------------+----+----+------------------------+------------+---------------------------------------------------+--------------------------+-----------------------------------+--------------------+
|         STATE/UT|         DISTRICT|Year|Rape|Kidnapping and Abduction|Dowry Deaths|Assault on women with intent to outrage her modesty|Insult to modesty of Women|Cruelty by Husband or his Relatives|Importation of Girls|
+-----------------+-----------------+----+----+------------------------+------------+---------------------------------------------------+--------------------------+-----------------------------------+--------------------+
|   ANDHRA PRADESH|    GUNTAKAL RLY.|2001|   0|                       0|           0|                                                  1|                         0|                                  0|                   0|
|   ANDHRA PRADESH|SECUNDERABAD RLY.|2001|   0|                       0|           1|                           

In [24]:
from pyspark.sql import functions as F

# Filter the DataFrame for 'Murder' column
murder_df = df.select('DISTRICT', 'Year', 'Rape')

# Group by 'DISTRICT' and 'Year' and sum up the murders
murder_count = murder_df.groupBy('DISTRICT', 'Year').agg(F.sum('Rape').alias('Total_Rape'))

# Sort the result in ascending order by 'DISTRICT' and 'Year'
sorted_murder_count = murder_count.orderBy('DISTRICT', 'Year')

# Show the result
sorted_murder_count.show()

+-----------------+----+----------+
|         DISTRICT|Year|Total_Rape|
+-----------------+----+----------+
|24 PARGANAS NORTH|2001|        53|
|24 PARGANAS NORTH|2002|        88|
|24 PARGANAS NORTH|2003|        92|
|24 PARGANAS NORTH|2004|       176|
|24 PARGANAS NORTH|2005|       217|
|24 PARGANAS NORTH|2006|       174|
|24 PARGANAS NORTH|2007|       231|
|24 PARGANAS NORTH|2008|       192|
|24 PARGANAS NORTH|2009|       166|
|24 PARGANAS NORTH|2010|       173|
|24 PARGANAS NORTH|2011|       203|
|24 PARGANAS NORTH|2012|       118|
|24 PARGANAS SOUTH|2001|        47|
|24 PARGANAS SOUTH|2002|        68|
|24 PARGANAS SOUTH|2003|        89|
|24 PARGANAS SOUTH|2004|       122|
|24 PARGANAS SOUTH|2005|       136|
|24 PARGANAS SOUTH|2006|       190|
|24 PARGANAS SOUTH|2007|       205|
|24 PARGANAS SOUTH|2008|       308|
+-----------------+----+----------+
only showing top 20 rows



3.4	Insert records from 02_District_wise_crimes_committed_against_ST_2001_2012.csv into a new table

In [32]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("CrimesData").getOrCreate()

# Define the path to your CSV file
csv_path = "C:/Users/Sayan Mondal/Downloads/India_Crime - Copy/India_Crime - Copy/02_District_wise_crimes_committed_against_ST_2001_2012.csv"

# Read the CSV data into a DataFrame (without header)
df1 = spark.read.csv(csv_path, header=False)

# Print a message to confirm successful reading
print(f"Data from '{csv_path}' successfully read into a DataFrame")

Data from 'C:/Users/Sayan Mondal/Downloads/India_Crime - Copy/India_Crime - Copy/02_District_wise_crimes_committed_against_ST_2001_2012.csv' successfully read into a DataFrame


In [33]:
df1.show()

+--------------+--------------+----+------+----+--------------------+-------+-------+-----+----+--------------------+--------------------+--------------------+
|           _c0|           _c1| _c2|   _c3| _c4|                 _c5|    _c6|    _c7|  _c8| _c9|                _c10|                _c11|                _c12|
+--------------+--------------+----+------+----+--------------------+-------+-------+-----+----+--------------------+--------------------+--------------------+
|      STATE/UT|      DISTRICT|Year|Murder|Rape|Kidnapping Abduction|Dacoity|Robbery|Arson|Hurt|Protection of Civ...|Prevention of atr...|Other Crimes Agai...|
|ANDHRA PRADESH|      ADILABAD|2001|     0|   1|                   2|      0|      0|    0|   2|                   0|                   0|                  13|
|ANDHRA PRADESH|     ANANTAPUR|2001|     0|   0|                   0|      0|      0|    0|   7|                   0|                   1|                   6|
|ANDHRA PRADESH|      CHITTOOR|2001|    

In [35]:
df1.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12']

In [36]:
# Delete the first row (column headings row)
df1 = df1.filter(df1["_c0"] != "column_heading_1")

# Set the first row as column names
new_columns = df1.first()
df1 = df1.filter(df1._c0 != new_columns[0])
df1 = df1.toDF(*new_columns)

# Show the resulting dataframe
df1.show()

+--------------+-----------------+----+------+----+--------------------+-------+-------+-----+----+------------------------------------+----------------------------------+------------------------+
|      STATE/UT|         DISTRICT|Year|Murder|Rape|Kidnapping Abduction|Dacoity|Robbery|Arson|Hurt|Protection of Civil Rights (PCR) Act|Prevention of atrocities (POA) Act|Other Crimes Against STs|
+--------------+-----------------+----+------+----+--------------------+-------+-------+-----+----+------------------------------------+----------------------------------+------------------------+
|ANDHRA PRADESH|         ADILABAD|2001|     0|   1|                   2|      0|      0|    0|   2|                                   0|                                 0|                      13|
|ANDHRA PRADESH|        ANANTAPUR|2001|     0|   0|                   0|      0|      0|    0|   7|                                   0|                                 1|                       6|
|ANDHRA PRADESH

In [37]:
df1.columns

['STATE/UT',
 'DISTRICT',
 'Year',
 'Murder',
 'Rape',
 'Kidnapping Abduction',
 'Dacoity',
 'Robbery',
 'Arson',
 'Hurt',
 'Protection of Civil Rights (PCR) Act',
 'Prevention of atrocities (POA) Act',
 'Other Crimes Against STs']

3.5	Write SQL query to find the highest number of dacoity/robbery in which district.

In [39]:
from pyspark.sql import functions as F

# Filter out the row with the district named "TOTAL"
filtered_df = df1.filter(F.col('DISTRICT') != 'TOTAL')

# Filter the DataFrame for 'Dacoity' and 'Robbery' columns
dacoity_robbery_df = filtered_df.select('DISTRICT', 'Dacoity', 'Robbery')

# Add a new column to calculate the total of 'Dacoity' and 'Robbery'
dacoity_robbery_df = dacoity_robbery_df.withColumn('Total_Dacoity_Robbery', F.col('Dacoity') + F.col('Robbery'))

# Find the district with the highest total of 'Dacoity' and 'Robbery'
highest_dacoity_robbery_district = dacoity_robbery_df.orderBy(F.col('Total_Dacoity_Robbery').desc()).select('DISTRICT').first()

print("District with the highest number of Dacoity or Robbery:", highest_dacoity_robbery_district[0])



District with the highest number of Dacoity or Robbery: DAHOD


3.6	Write SQL query to find in which districts(All) the lowest number of murders happened

In [40]:
from pyspark.sql import functions as F

# Select the necessary columns from the DataFrame
murder_df = df1.select('DISTRICT', 'Murder')

# Group by 'DISTRICT' and find the minimum number of murders
min_murders_by_district = murder_df.groupBy('DISTRICT').agg(F.min('Murder').alias('Min_Murders'))

# Sort the result in ascending order by the minimum number of murders
sorted_min_murders = min_murders_by_district.orderBy('Min_Murders')

# Show the districts with the lowest number of murders
sorted_min_murders.show()


+-----------------+-----------+
|         DISTRICT|Min_Murders|
+-----------------+-----------+
|24 PARGANAS NORTH|          0|
|24 PARGANAS SOUTH|          0|
|  A and N ISLANDS|          0|
|         ADILABAD|          0|
|             AGRA|          0|
| AHMEDABAD COMMR.|          0|
|  AHMEDABAD RURAL|          0|
|       AHMEDNAGAR|          0|
|        AHWA-DANG|          0|
|           AIZAWL|          0|
|            AJMER|          0|
|            AKOLA|          0|
|         ALAPUZHA|          0|
|          ALIGARH|          0|
|        ALIRAJPUR|          0|
|        ALLAHABAD|          0|
|           ALMORA|          0|
|            ALWAR|          0|
|           AMBALA|          0|
|     AMBALA RURAL|          0|
+-----------------+-----------+
only showing top 20 rows



3.7	Write SQL query to find the number of murders in ascending order in district and yearwise.

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MurderAnalysis") \
    .getOrCreate()

# Assuming your DataFrame is named df1
# Select the necessary columns for analysis
murder_df = df1.select('STATE/UT', 'DISTRICT', 'Year', 'Murder')

# Group by 'DISTRICT' and 'Year' and sum up the murders
murder_count = murder_df.groupBy('DISTRICT', 'Year').agg(F.sum('Murder').alias('Total_Murders'))

# Sort the result in ascending order by 'DISTRICT' and 'Year'
sorted_murder_count = murder_count.orderBy('DISTRICT', 'Year')

# Show the result
sorted_murder_count.show()


+-----------------+----+-------------+
|         DISTRICT|Year|Total_Murders|
+-----------------+----+-------------+
|24 PARGANAS NORTH|2001|          0.0|
|24 PARGANAS NORTH|2002|          0.0|
|24 PARGANAS NORTH|2003|          0.0|
|24 PARGANAS NORTH|2004|          0.0|
|24 PARGANAS NORTH|2005|          0.0|
|24 PARGANAS NORTH|2006|          0.0|
|24 PARGANAS NORTH|2007|          0.0|
|24 PARGANAS NORTH|2008|          0.0|
|24 PARGANAS NORTH|2009|          0.0|
|24 PARGANAS NORTH|2010|          0.0|
|24 PARGANAS NORTH|2011|          0.0|
|24 PARGANAS NORTH|2012|          0.0|
|24 PARGANAS SOUTH|2001|          0.0|
|24 PARGANAS SOUTH|2002|          0.0|
|24 PARGANAS SOUTH|2003|          0.0|
|24 PARGANAS SOUTH|2004|          0.0|
|24 PARGANAS SOUTH|2005|          0.0|
|24 PARGANAS SOUTH|2006|          0.0|
|24 PARGANAS SOUTH|2007|          0.0|
|24 PARGANAS SOUTH|2008|          0.0|
+-----------------+----+-------------+
only showing top 20 rows



3.8.1	Insert records of STATE/UT, DISTRICT, YEAR, MURDER, ATTEMPT TO MURDER, and RAPE columns only from 01_District_wise_crimes_committed_IPC_2001_2012.csv into a new table

In [31]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("CrimesData").getOrCreate()

# Define the path to your CSV file
csv_path = "C:/Users/Sayan Mondal/Downloads/India_Crime - Copy/India_Crime - Copy/01_District_wise_crimes_committed_IPC_2001_2012.csv"

# Read the CSV data into a DataFrame (without header)
df2 = spark.read.csv(csv_path, header=False)

# Print a message to confirm successful reading
print(f"Data from '{csv_path}' successfully read into a DataFrame")

Data from 'C:/Users/Sayan Mondal/Downloads/India_Crime - Copy/India_Crime - Copy/01_District_wise_crimes_committed_IPC_2001_2012.csv' successfully read into a DataFrame


In [9]:
df2.show()

+--------------+--------------+----+------+-----------------+--------------------+----+--------------+----------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+-----+----------+-----------+-----+--------------------+--------+--------------+-----+------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------+
|           _c0|           _c1| _c2|   _c3|              _c4|                 _c5| _c6|           _c7|       _c8|                 _c9|                _c10|                _c11|   _c12|                _c13|   _c14|    _c15| _c16|      _c17|       _c18| _c19|                _c20|    _c21|          _c22| _c23|              _c24|        _c25|                _c26|                _c27|                _c28|                _c29|                _c30|            _c31|            _c32|
+--------------+--------------+----+----

In [10]:
df2.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29',
 '_c30',
 '_c31',
 '_c32']

In [11]:
# Delete the first row (column headings row)
df2 = df2.filter(df2["_c0"] != "column_heading_1")

# Set the first row as column names
new_columns = df2.first()
df2 = df2.filter(df2._c0 != new_columns[0])
df2 = df2.toDF(*new_columns)

# Show the resulting dataframe
df2.show()

+--------------+-----------------+----+------+-----------------+-----------------------------------------+----+--------------+----------+----------------------+-------------------------------------------+----------------------------------+-------+------------------------------------+-------+--------+-----+----------+-----------+-----+------------------------+--------+--------------+-----+------------------+------------+---------------------------------------------------+--------------------------+-----------------------------------+-------------------------------------------+---------------------------+----------------+----------------+
|      STATE/UT|         DISTRICT|YEAR|MURDER|ATTEMPT TO MURDER|CULPABLE HOMICIDE NOT AMOUNTING TO MURDER|RAPE|CUSTODIAL RAPE|OTHER RAPE|KIDNAPPING & ABDUCTION|KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS|KIDNAPPING AND ABDUCTION OF OTHERS|DACOITY|PREPARATION AND ASSEMBLY FOR DACOITY|ROBBERY|BURGLARY|THEFT|AUTO THEFT|OTHER THEFT|RIOTS|CRIMINAL BREACH OF 

In [12]:
df2.columns

['STATE/UT',
 'DISTRICT',
 'YEAR',
 'MURDER',
 'ATTEMPT TO MURDER',
 'CULPABLE HOMICIDE NOT AMOUNTING TO MURDER',
 'RAPE',
 'CUSTODIAL RAPE',
 'OTHER RAPE',
 'KIDNAPPING & ABDUCTION',
 'KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS',
 'KIDNAPPING AND ABDUCTION OF OTHERS',
 'DACOITY',
 'PREPARATION AND ASSEMBLY FOR DACOITY',
 'ROBBERY',
 'BURGLARY',
 'THEFT',
 'AUTO THEFT',
 'OTHER THEFT',
 'RIOTS',
 'CRIMINAL BREACH OF TRUST',
 'CHEATING',
 'COUNTERFIETING',
 'ARSON',
 'HURT/GREVIOUS HURT',
 'DOWRY DEATHS',
 'ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY',
 'INSULT TO MODESTY OF WOMEN',
 'CRUELTY BY HUSBAND OR HIS RELATIVES',
 'IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES',
 'CAUSING DEATH BY NEGLIGENCE',
 'OTHER IPC CRIMES',
 'TOTAL IPC CRIMES']

3.8.1	Insert records of STATE/UT, DISTRICT, YEAR, MURDER, ATTEMPT TO MURDER, and RAPE columns only from 01_District_wise_crimes_committed_IPC_2001_2012.csv into a new table

CHECKING COLUMNS TO VERIFY BEFORE QUERY

In [13]:
new_df = df2.select("STATE/UT", "DISTRICT", "YEAR", "MURDER", "ATTEMPT TO MURDER", "RAPE")

new_df.show()

+--------------+-----------------+----+------+-----------------+----+
|      STATE/UT|         DISTRICT|YEAR|MURDER|ATTEMPT TO MURDER|RAPE|
+--------------+-----------------+----+------+-----------------+----+
|ANDHRA PRADESH|         ADILABAD|2001|   101|               60|  50|
|ANDHRA PRADESH|        ANANTAPUR|2001|   151|              125|  23|
|ANDHRA PRADESH|         CHITTOOR|2001|   101|               57|  27|
|ANDHRA PRADESH|         CUDDAPAH|2001|    80|               53|  20|
|ANDHRA PRADESH|    EAST GODAVARI|2001|    82|               67|  23|
|ANDHRA PRADESH|    GUNTAKAL RLY.|2001|     3|                1|   0|
|ANDHRA PRADESH|           GUNTUR|2001|   182|               88|  54|
|ANDHRA PRADESH|   HYDERABAD CITY|2001|   111|              113|  37|
|ANDHRA PRADESH|       KARIMNAGAR|2001|   162|               85|  56|
|ANDHRA PRADESH|          KHAMMAM|2001|    93|               60|  47|
|ANDHRA PRADESH|          KRISHNA|2001|    65|               51|  37|
|ANDHRA PRADESH|    

RECHECKING TOTAL COLUMNS

In [14]:
new_df.columns

['STATE/UT', 'DISTRICT', 'YEAR', 'MURDER', 'ATTEMPT TO MURDER', 'RAPE']

3.8.2	Write SQL query to find which District in each state/ut has the highest number of murders yearwise. Your output should show STATE/UT, YEAR, DISTRICT, and MURDERS.

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define a window specification partitioned by STATE/UT and YEAR, and ordered by MURDER in descending order
window_spec = Window.partitionBy("STATE/UT", "YEAR").orderBy(F.col("MURDER").desc())

# Add a row number column to each partition
new_df_with_row_number = new_df.withColumn("rn", F.row_number().over(window_spec))

# Filter the rows where the row number is 1 (i.e., the row with the highest number of murders in each partition)
district_with_highest_murder = new_df_with_row_number.filter(F.col("rn") == 1)

# Select the required columns
result = district_with_highest_murder.select("STATE/UT", "DISTRICT", "YEAR", "MURDER")

# Show the result
result.show()

+--------------+---------------+----+------+
|      STATE/UT|       DISTRICT|YEAR|MURDER|
+--------------+---------------+----+------+
| A & N ISLANDS|        ANDAMAN|2001|    13|
| A & N ISLANDS|        NICOBAR|2002|     3|
| A & N ISLANDS|        NICOBAR|2003|     5|
| A & N ISLANDS|        NICOBAR|2004|     2|
| A & N ISLANDS|        ANDAMAN|2005|    14|
| A & N ISLANDS|          TOTAL|2006|     4|
| A & N ISLANDS|A and N ISLANDS|2007|    15|
| A & N ISLANDS|        ANDAMAN|2008|     8|
| A & N ISLANDS|        ANDAMAN|2009|    15|
| A & N ISLANDS|          TOTAL|2010|     9|
| A & N ISLANDS|        ANDAMAN|2011|    14|
| A & N ISLANDS|          SOUTH|2012|     8|
|ANDHRA PRADESH|        KHAMMAM|2001|    93|
|ANDHRA PRADESH|  EAST GODAVARI|2002|    97|
|ANDHRA PRADESH|       CUDDAPAH|2003|    95|
|ANDHRA PRADESH|       CUDDAPAH|2004|    96|
|ANDHRA PRADESH|       CUDDAPAH|2005|    96|
|ANDHRA PRADESH|       ADILABAD|2006|    99|
|ANDHRA PRADESH|  EAST GODAVARI|2007|    99|
|ANDHRA PR

3.8.3	Store the above data (the result of 3.2) in DataFrame and analyze districts that appear 3 or more than 3 years and print the corresponding state/ut, district, murders, and year in descending order.

In [16]:
from pyspark.sql import functions as F

# Filter the DataFrame for 'Murder' column
murder_df = df.select('DISTRICT', 'Year', 'Rape')

# Group by 'DISTRICT' and 'Year' and sum up the murders
murder_count = murder_df.groupBy('DISTRICT', 'Year').agg(F.sum('Rape').alias('Total_Rape'))

# Count the number of years each district appears
district_year_count = murder_count.groupBy('DISTRICT').agg(F.countDistinct('Year').alias('Years'))

# Filter districts that appear 3 or more than 3 years
districts_to_analyze = district_year_count.filter('Years >= 3')

# Join the filtered districts with the murder_count DataFrame to get the corresponding state/UT and year
result = districts_to_analyze.join(murder_count, on='DISTRICT', how='inner')

# Show the result sorted by state/UT, district, murders, and year in descending order
result.orderBy('DISTRICT', 'Total_Rape', 'Year', ascending=[True, False, False]).show()

+-----------------+-----+----+----------+
|         DISTRICT|Years|Year|Total_Rape|
+-----------------+-----+----+----------+
|24 PARGANAS NORTH|   12|2007|       231|
|24 PARGANAS NORTH|   12|2005|       217|
|24 PARGANAS NORTH|   12|2011|       203|
|24 PARGANAS NORTH|   12|2008|       192|
|24 PARGANAS NORTH|   12|2004|       176|
|24 PARGANAS NORTH|   12|2006|       174|
|24 PARGANAS NORTH|   12|2010|       173|
|24 PARGANAS NORTH|   12|2009|       166|
|24 PARGANAS NORTH|   12|2012|       118|
|24 PARGANAS NORTH|   12|2003|        92|
|24 PARGANAS NORTH|   12|2002|        88|
|24 PARGANAS NORTH|   12|2001|        53|
|24 PARGANAS SOUTH|   12|2008|       308|
|24 PARGANAS SOUTH|   12|2009|       280|
|24 PARGANAS SOUTH|   12|2011|       274|
|24 PARGANAS SOUTH|   12|2010|       258|
|24 PARGANAS SOUTH|   12|2012|       244|
|24 PARGANAS SOUTH|   12|2007|       205|
|24 PARGANAS SOUTH|   12|2006|       190|
|24 PARGANAS SOUTH|   12|2005|       136|
+-----------------+-----+----+----

3.8.4	Use appropriate graphs to show your data (the result of 3.8.3)

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# Plot the total number of rapes for each district
plt.figure(figsize=(12, 6))
plt.bar(result['DISTRICT'], result['Total_Rape'], color='skyblue')
plt.xlabel('District')
plt.ylabel('Total Rapes')
plt.title('Total Rapes in Districts Appearing 3 or More Years')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Total_Rape` cannot be resolved. Did you mean one of the following? [`STATE/UT`, `DISTRICT`, `Year`, `Rape`, `Kidnapping and Abduction`].

<Figure size 1200x600 with 0 Axes>